In [65]:
import os
import pandas as pd
import numpy as np

In [152]:
wkdir = '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/3_Scribe'
os.chdir(wkdir)

### Read TF list from reference articles

In [133]:
tstem_mtf_file = '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/z_References/Tstem_PNAS/Motifs.csv'
runx3_mtf_file = '/media/pipkin/Yolanda/Exp391_Acute-Chronic_SC/z_References/Runx3_Immunity/TopMotifs.csv'
tstem_mtf_df = pd.read_csv(tstem_mtf_file)
runx3_mtf_df = pd.read_csv(runx3_mtf_file)

tstem_mtfs = [x.split("(")[0] for x in tstem_mtf_df['Motif']]
runx3_mtfs = [x.split("(")[0] for x in runx3_mtf_df['Motif']]

all_mtfs = list(set(tstem_mtfs + runx3_mtfs))
all_mtfs = [x.split(") ")[-1] for x in all_mtfs]
all_mtfs = "|".join(all_mtfs).replace("−", "|").replace("-", "|").replace(":", "|").lower()
all_mtfs = list(set(all_mtfs.split("|")))

all_mtfs[:10]

['mafa',
 'bcl6',
 'oct',
 'batf',
 'gsc',
 'fosl2',
 'rfx',
 'halfsite',
 'ap1',
 'etv2']

## Read TF class database info

In [131]:
tfclass_db = "https://raw.githubusercontent.com/Yolanda-HT/TFclassDataCollection/master/MouseTFclassification_noISO_gn_alt.csv"
tfclass_df = pd.read_csv(tfclass_db)
tfclass_df.head()

###----- Build consensus motif dataframe
isMotifs = [True if str(x) != 'nan' else False for x in tfclass_df['ConsensusMotif'].values]
motifs_df = tfclass_df[isMotifs]

###----- Remove all the redudant consensus motifs
index_all = "|".join(motifs_df['Index'])
nonredundant_index = [False if "%s."%x in index_all else True for x in motifs_df['Index'].values]
motifs_df = motifs_df[nonredundant_index]

###----- Find all gene names associated with a certain motif
motifs_df_gnames = list(motifs_df['Names'])
motifs_df_indexes = list(motifs_df['Index'])
tfclass_df_indexes = list(tfclass_df['Index'])
gnames = []
for row_i in range(0, len(motifs_df)):
    gname_i = []
    index_i = motifs_df_indexes[row_i]
    if str(motifs_df_gnames[row_i]) != 'nan':
        gname_i.append(motifs_df_gnames[row_i])
    use_idx = [True if x.startswith("%s."%index_i) else False for x in tfclass_df_indexes ]
    row_i_df = tfclass_df[use_idx]
    row_i_df_gn = [True if str(x) != 'nan' else False for x in row_i_df['Names']]
    row_i_df = row_i_df[row_i_df_gn]
    if len(row_i_df) > 0:
        gname_i += ", ".join(row_i_df['Names']).split(", ")
        gnames.append(gname_i)
    else:
        gnames.append([])
gnames = ["|".join(x) for x in gnames]
motifs_df['Names'] = gnames

In [142]:
###----- Find all tf names with same consensus motifs as tfs in the list
alternative_tfs = [x for x in motifs_df['Names'].values if np.any(np.in1d(np.array(x.split("|")), np.array(all_mtfs)))]

## Generate gene list

In [153]:
all_genes = all_mtfs + "|".join(alternative_tfs).split("|")
all_genes = list(set(all_genes))
all_genes = [x.capitalize() for x in all_genes]
all_genes_df = pd.DataFrame({"gene_name": all_genes})
all_genes_df.to_csv("use_TF_list.csv")